In [104]:
import pandas as pd
import ast
import re
import json

In [105]:
# Import sample data
sample_df = pd.read_parquet('project_b_samples_2k.parquet')
sample_df

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_sources,base_names,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses
0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Goin' Postal Jacksonville""}","{""primary"":""shipping_center"",""alternate"":[""fre...",0.996261,"[""http://www.goinpostaljacksonville.com/""]","[""https://www.facebook.com/463273470392736""]",None,"[""+19049989600""]",...,"[{""dataset"":""Microsoft"",""record_id"":""168884986...","{""primary"":""Goin' Postal Jacksonville""}","{""primary"":""vehicle_shipping"",""alternate"":[""co...",0.77,"[""https://www.goinpostaljacksonville.com/""]",None,None,"[""9049989600""]","{""names"":{}}","[{""freeform"":""7643 Gate Pkwy Ste 104"",""localit..."
1,08f29a456e42e5830324637954145c50,1125899907111860,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Valley Transmission""}","{""primary"":""automotive_repair"",""alternate"":[""a...",0.996261,"[""http://valleytransca.com/""]","[""https://www.facebook.com/107150256087765""]",None,"[""+16194474353""]",...,"[{""dataset"":""Microsoft"",""record_id"":""112589990...","{""primary"":""Valley Transmission""}","{""primary"":""transmission_repair"",""alternate"":[...",0.77,"[""http://valleytransca.com/""]",None,None,"[""6194474353""]","{""names"":{}}","[{""freeform"":""1158 N 2nd St"",""locality"":""El Ca..."
2,08fbcd0030da5323031bcafa8c2fa0dc,844424934845986,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Mazda Nelspruit""}","{""primary"":""car_dealer"",""alternate"":[""automoti...",0.997899,"[""http://bit.ly/NelspruitMazda""]","[""https://www.facebook.com/113070635372645""]",None,"[""+27137573800""]",...,"[{""dataset"":""Microsoft"",""record_id"":""844424934...","{""primary"":""Mazda Nelspruit""}","{""primary"":""used_car_dealer"",""alternate"":[""aut...",0.77,"[""https://nelspruitmazda.co.za/""]",None,None,"[""0137573800""]","{""names"":{}}","[{""freeform"":""49 Emnotweni Avenue"",""locality"":..."
3,1407374885933937,844424932146366,"[{""dataset"":""Microsoft"",""record_id"":""140737488...","{""primary"":""Red Wing - Roswell, GA""}","{""primary"":""shoe_store"",""alternate"":[""online_s...",0.770000,"[""https://stores.redwingshoes.com/roswell-ga""]",None,None,"[""7705584400""]",...,"[{""dataset"":""Microsoft"",""record_id"":""844424932...","{""primary"":""Red Wing""}","{""primary"":""shoe_store"",""alternate"":[""online_s...",0.77,"[""http://stores.redwing.com/roswell-ga""]",None,None,"[""7705584400""]","{""names"":{}}","[{""freeform"":""875 Mansell Rd Ste B1"",""locality..."
4,08f3956260b9e14003feca2bf0764d0c,1407374887472291,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Norauto España""}","{""primary"":""automotive_repair"",""alternate"":[""c...",0.996261,"[""https://centro.norauto.es/alacant/finestrat/...","[""https://www.facebook.com/295967644448975""]",None,"[""+34966803254""]",...,"[{""dataset"":""Microsoft"",""record_id"":""140737488...","{""primary"":""Norauto""}","{""primary"":""brake_service_and_repair"",""alterna...",0.77,"[""http://centro.norauto.es/centre/alicante-fin...",None,None,"[""966803254""]","{""names"":{}}","[{""freeform"":""Calle Rei Jaume I 9"",""locality"":..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,08f1e1251b0a66c003a195f0bf6b4bc9,502a184a011ce8eabd5934fb,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Studio Legale Associato Petracci M...","{""primary"":""lawyer"",""alternate"":[""criminal_def...",0.286713,"[""http://www.petraccimarin.it""]","[""https://www.facebook.com/121776774503515""]",None,"[""+39040660022""]",...,"[{""property"":"""",""dataset"":""FourSquare"",""record...","{""primary"":""Studio Legale Associato Petracci M...","{""primary"":""Business 

In [106]:
# For previewing the data, look at first data point in sample_df
first_data_point = sample_df.iloc[0]

print("Sources value:")
print(first_data_point['names'])
print(type(first_data_point['names']))

print("\nParsed sources:")
sources_data = json.loads(first_data_point['names'])
for i, source in enumerate(sources_data):
    print(f"Source {i+1}: {source}")

Sources value:
{"primary":"Goin' Postal Jacksonville"}
<class 'str'>

Parsed sources:
Source 1: primary


In [107]:
# Load the Yelp business dataset
with open('Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json', 'r') as file:
    businesses = []
    for line in file:
        business = json.loads(line.strip())
        businesses.append(business)

print(f"Number of records: {len(businesses)}")
print("Sample business record:")
print(json.dumps(businesses[0], indent=2))

Number of records: 150346
Sample business record:
{
  "business_id": "Pns2l4eNsfO8kk83dixA6A",
  "name": "Abby Rappoport, LAC, CMQ",
  "address": "1616 Chapala St, Ste 2",
  "city": "Santa Barbara",
  "state": "CA",
  "postal_code": "93101",
  "latitude": 34.4266787,
  "longitude": -119.7111968,
  "stars": 5.0,
  "review_count": 7,
  "is_open": 0,
  "attributes": {
    "ByAppointmentOnly": "True"
  },
  "categories": "Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists",
  "hours": null
}


In [108]:
yelp_df = pd.DataFrame(businesses)

print(f"DataFrame shape: {yelp_df.shape}")
print("\nDataFrame columns:")
print(yelp_df.columns.tolist())
yelp_df

DataFrame shape: (150346, 14)

DataFrame columns:
['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours']


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [109]:
# Check current types
sample_df.map(type)

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_sources,base_names,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses
0,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
1,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
2,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
3,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
4,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1996,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1997,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'NoneType'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1998,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [110]:
# Assigns data types to attributes
def parse_category(x):
    if isinstance(x, (list, dict)) or pd.isna(x): return x
    try: return ast.literal_eval(x)
    except: return None

for c in ["sources", "names", "addresses","base_names","base_addresses","base_categories", "categories", "websites", "brand"]:
    if c in sample_df: sample_df[c] = sample_df[c].apply(parse_category)

sample_df.map(type)

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_sources,base_names,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses
0,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
1,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
2,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
3,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
4,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>
1996,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>
1997,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'NoneType'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>
1998,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>


## Normalize and Extract Important Information
Let's create some helper functions to normalize names, and extract crucial address information

In [111]:
def norm_name(s):
    s = re.sub(r"\s+", " ", (s or "").strip().lower())
    s = re.sub(r"\b(llc|inc|corp|co|ltd|the)\b", "", s)
    s = re.sub(r"[^a-z0-9\s&\-']", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def extract_postcode(addr_list):
    if isinstance(addr_list, list) and addr_list:
        pc = str(addr_list[0].get("postcode") or "")
        return pc

def extract_city(addr_list):
    if isinstance(addr_list, list) and addr_list:
        return (addr_list[0].get("locality") or "").strip().lower()

def extract_name(name_list):
    if isinstance(name_list, dict) and name_list:
        return name_list.get("primary")

In [112]:
sample_df["name_candidate"] = sample_df["names"].apply(extract_name)
sample_df["name_norm"] = sample_df["name_candidate"].fillna("").apply(norm_name)
sample_df["zip"] = sample_df["base_addresses"].apply(extract_postcode)
sample_df["city_norm"] = sample_df["base_addresses"].apply(extract_city)

In [113]:
sample_df

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses,name_candidate,name_norm,zip,city_norm
0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Goin' Postal Jacksonville'},"{'primary': 'shipping_center', 'alternate': ['...",0.996261,[http://www.goinpostaljacksonville.com/],"[""https://www.facebook.com/463273470392736""]",None,"[""+19049989600""]",...,"[""https://www.goinpostaljacksonville.com/""]",None,None,"[""9049989600""]","{""names"":{}}","[{'freeform': '7643 Gate Pkwy Ste 104', 'local...",Goin' Postal Jacksonville,goin' postal jacksonville,32256,jacksonville
1,08f29a456e42e5830324637954145c50,1125899907111860,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Valley Transmission'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[http://valleytransca.com/],"[""https://www.facebook.com/107150256087765""]",None,"[""+16194474353""]",...,"[""http://valleytransca.com/""]",None,None,"[""6194474353""]","{""names"":{}}","[{'freeform': '1158 N 2nd St', 'locality': 'El...",Valley Transmission,valley transmission,92021,el cajon
2,08fbcd0030da5323031bcafa8c2fa0dc,844424934845986,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Mazda Nelspruit'},"{'primary': 'car_dealer', 'alternate': ['autom...",0.997899,[http://bit.ly/NelspruitMazda],"[""https://www.facebook.com/113070635372645""]",None,"[""+27137573800""]",...,"[""https://nelspruitmazda.co.za/""]",None,None,"[""0137573800""]","{""names"":{}}","[{'freeform': '49 Emnotweni Avenue', 'locality...",Mazda Nelspruit,mazda nelspruit,1201,mbombela
3,1407374885933937,844424932146366,"[{'dataset': 'Microsoft', 'record_id': '140737...","{'primary': 'Red Wing - Roswell, GA'}","{'primary': 'shoe_store', 'alternate': ['onlin...",0.770000,[https://stores.redwingshoes.com/roswell-ga],None,None,"[""7705584400""]",...,"[""http://stores.redwing.com/roswell-ga""]",None,None,"[""7705584400""]","{""names"":{}}","[{'freeform': '875 Mansell Rd Ste B1', 'locali...","Red Wing - Roswell, GA",red wing - roswell ga,30076,roswell
4,08f3956260b9e14003feca2bf0764d0c,1407374887472291,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Norauto España'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[https://centro.norauto.es/alacant/finestrat/c...,"[""https://www.facebook.com/295967644448975""]",None,"[""+34966803254""]",...,"[""http://centro.norauto.es/centre/alicante-fin...",None,None,"[""966803254""]","{""names"":{}}","[{'freeform': 'Calle Rei Jaume I 9', 'locality...",Norauto España,norauto espa a,03509,alicante
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,08f1e1251b0a66c003a195f0bf6b4bc9,502a184a011ce8eabd5934fb,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Studio Legale Associato Petracci ...,"{'primary': 'lawyer', 'alternate': ['criminal_...",0.286713,[http://www.petraccimarin.it],"[""https://www.facebook.com/121776774503515""]",None,"[""+39040660022""]",...,"[""http://www.petraccimarin.it""]","[null,null,null]",[null],"[""040 660022""]","{""wikidata"":"""",""names"":{""primary"":""Studio Lega...","[{'freeform': 'Via beccaria 3', 'locality': 'T...",Studio Legale Associato Petracci Marin,studio legale associato petracci marin,34125,trieste
1996,08fa80b035674b6a03379ca9c1ecea9f,50364256e4b0573431043dd4,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Farmácias Pague Menos'},"{'primary': 'pharmacy', 'alternate': ['pharmac...",0.792793,[https://www.paguemenos.com.br/],"[""https://www.facebook.com/2204406756483378""]",None,"[""+556740028282""]",...,"[""https://www.paguemenos.com.br""]","[null,null,null]",[null],"[""(67) 3521-4016""]","{""wikidata"":"""",""names"":{""primary"":""Farmacias P...","[{'freeform': 'Av. Antônio Trajano, 583', 'loc...",Farmácias Pague Menos,farm cias pague menos,,três la

In [114]:
l = sample_df['addresses'].apply(len)
max_length = l.max()
print(max_length)


1
